Import Libraries


In [1]:
import os
from langchain_community.llms import Ollama
from dotenv import load_dotenv
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain


Loading The LLM (Language Model)

In [2]:
llm = Ollama(model="llama3.2:3b", base_url="http://172.22.54.171:11434")

C:\Users\Tzahi\AppData\Local\Temp\ipykernel_8924\2680234934.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2:3b", base_url="http://172.22.54.171:11434")


Setting Ollama Embeddings

In [3]:
embed_model = OllamaEmbeddings(
    model="llama3.2:3b",
    base_url='http://172.22.54.171:11434'
)

C:\Users\Tzahi\AppData\Local\Temp\ipykernel_8924\4262251034.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embed_model = OllamaEmbeddings(


Loading Text

In [4]:
text = """
    In the lush canopy of a tropical rainforest, two mischievous monkeys, Coco and Mango, swung from branch to branch, their playful antics echoing through the trees. They were inseparable companions, sharing everything from juicy fruits to secret hideouts high above the forest floor. One day, while exploring a new part of the forest, Coco stumbled upon a beautiful orchid hidden among the foliage. Entranced by its delicate petals, Coco plucked it and presented it to Mango with a wide grin. Overwhelmed by Coco's gesture of friendship, Mango hugged Coco tightly, cherishing the bond they shared. From that day on, Coco and Mango ventured through the forest together, their friendship growing stronger with each passing adventure. As they watched the sun dip below the horizon, casting a golden glow over the treetops, they knew that no matter what challenges lay ahead, they would always have each other, and their hearts brimmed with joy.
    """

Splitting Text into Chunks

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
chunks = text_splitter.split_text(text)

Creating a Vector Store (Chroma) from Text

In [7]:
vector_store = Chroma.from_texts(chunks, embed_model)

Creating a Retriever

In [8]:
retriever = vector_store.as_retriever()

Creating a Retrieval Chain


In [9]:
chain = create_retrieval_chain(combine_docs_chain=llm,retriever=retriever)

Retrieval-QA Chat Prompt


In [10]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

c:\Users\Tzahi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Combining Documents

In [11]:
combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

Final Retrieval Chain


In [12]:
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)    

Invoking the Retrieval Chain

In [13]:
response = retrieval_chain.invoke({"input": "Tell me name of monkeys and where do they live"})
print(response['answer'])

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


The two mischievous monkeys are named Coco and Mango. They live in a tropical rainforest.
